In [1]:
import os
import gc
import sys
import glob
import math
import numpy as np
import uproot
import pandas
from functools import partial
from concurrent.futures import ThreadPoolExecutor

import keras
import tensorflow as tf
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Conv2D, Dropout, AlphaDropout, Activation, BatchNormalization, Flatten
from keras.layers.merge import Concatenate
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras_tqdm import TQDMNotebookCallback

sys.path.insert(0, "../../python")
from common import *

Using TensorFlow backend.


In [2]:
class DataLoader:
    @staticmethod
    def GetNumberOfEntries(file_name, tree_name):
        with uproot.open(file_name) as file:
            tree = file[tree_name]
            return tree.numentries

    def __init__(self, file_name_pattern, batch_size, n_cells, has_validation_set):
        all_files = [ f.replace('\\', '/') for f in sorted(glob.glob(file_name_pattern)) ]
        if (has_validation_set and len(all_files) < 2) or len(all_files) < 1:
            raise RuntimeError("Too few input files")
        if batch_size <= 0:
            raise RuntimeError("batch_size should be a positive number")
        self.batch_size = batch_size
        self.n_cells = n_cells
        first_data_file_index = 0
        if has_validation_set:
            self.validation_files = all_files[0:1]
            self.validation_size = DataLoader.GetNumberOfEntries(self.validation_files[0], 'taus')
            first_data_file_index = 1
        self.data_files = all_files[first_data_file_index:]
        self.data_file_sizes = np.zeros(len(self.data_files), dtype=int)
        for n in range(len(self.data_files)):
            self.data_file_sizes[n] = DataLoader.GetNumberOfEntries(self.data_files[n], 'taus')
            if self.data_file_sizes[n] < batch_size:
                raise RuntimeError("File {} has too few events.".format(self.data_files[n]))
        self.data_size = self.data_file_sizes.sum()
        self.steps_per_epoch = math.floor(self.data_size / self.batch_size)
        if has_validation_set:
            self.validation_steps = math.floor(self.validation_size / self.batch_size)
        self.executor = ThreadPoolExecutor(12)
        
        
    def generator(self, primary_set):
        if primary_set:
            files = self.data_files
        else:
            files = self.validation_files

        def df_taus_gen():
            for df_taus in uproot.iterate(files, 'taus', df_tau_branches,
                                          entrysteps=self.batch_size, executor=self.executor):
                
                #df_taus.columns = [ c.decode('utf-8') for c in df_taus.columns ]
                yield df_taus

        def df_cells_gen():
            for df_cells in uproot.iterate(files, 'cells', df_cell_branches,
                                          entrysteps=self.batch_size * self.n_cells, executor=self.executor):
                #df_cells.columns = [ c.decode('utf-8') for c in df_cells.columns ]
                yield df_cells

        pfCand_branches = input_cell_common_branches + input_cell_pfCand_branches
        ele_branches = input_cell_common_branches + input_cell_ele_branches
        muon_branches = input_cell_common_branches + input_cell_muon_branches
        
        min_value, max_value = -1000, 1000
        
        
        while True:
            for df_taus, df_cells in zip(df_taus_gen(), df_cells_gen()):
                if df_taus[b'tau_pt'].shape[0] != self.batch_size: continue
                
                X_taus = np.empty((self.batch_size, len(input_tau_branches)))
                Y = np.empty((self.batch_size, n_outputs), dtype=int)
                weights = np.empty(self.batch_size)
                
                for n in range(len(input_tau_branches)):
                    X_taus[:, n] = df_taus[input_tau_branches[n].encode()].clip(min_value, max_value)
                
                for n in range(len(truth_branches)):
                    Y[:, n] = df_taus[truth_branches[n].encode()]
                
                weights[:] = df_taus[weight_branches[0].encode()]
                weights = weights / np.sum(weights) * weights.shape[0]
                
                X_cells_pfCand = np.empty((self.batch_size, n_cells_eta, n_cells_phi, len(pfCand_branches)))
                X_cells_ele = np.empty((self.batch_size, n_cells_eta, n_cells_phi, len(ele_branches)))
                X_cells_muon = np.empty((self.batch_size, n_cells_eta, n_cells_phi, len(muon_branches)))
                
                for n in range(len(pfCand_branches)):
                    X_cells_pfCand[:, :, :, n] = \
                        df_cells[pfCand_branches[n].encode()].clip(min_value, max_value) \
                        .reshape(self.batch_size, n_cells_eta, n_cells_phi)

                for n in range(len(ele_branches)):
                    X_cells_ele[:, :, :, n] = \
                        df_cells[ele_branches[n].encode()].clip(min_value, max_value) \
                        .reshape(self.batch_size, n_cells_eta, n_cells_phi)

                for n in range(len(muon_branches)):
                    X_cells_muon[:, :, :, n] = \
                        df_cells[muon_branches[n].encode()].clip(min_value, max_value) \
                        .reshape(self.batch_size, n_cells_eta, n_cells_phi)

                yield ([ X_taus, X_cells_pfCand, X_cells_ele, X_cells_muon ], Y, weights)
                

In [3]:
X_taus_shape=(44, )
X_cells_pfCand_shape = (13, 13, 31)
X_cells_ele_shape = (13, 13, 39)
X_cells_muon_shape = (13, 13, 39)

In [4]:
def conv_block(prev_layer, filters, kernel_size, activation, dropout_rate, block_name, n):
    conv = Conv2D(filters, kernel_size, name="{}_conv_{}".format(block_name, n),
                  kernel_initializer='he_uniform')(prev_layer)
    norm_layer = BatchNormalization(name="{}_batch_norm_{}".format(block_name, n))(conv)
    activation_layer = Activation(activation, name="{}_activation_{}".format(block_name, n))(norm_layer)
    if dropout_rate > 0:
        return Dropout(dropout_rate, name="{}_dropout_{}".format(block_name, n))(activation_layer)
    return activation_layer


def reduce_n_features_2d(input_layer, first_layer_size, last_layer_size, decay_factor, block_name, activation, dropout_rate):
    conv_kernel=(1, 1)
    prev_layer = input_layer
    current_size = first_layer_size
    n = 1
    while True:
        prev_layer = conv_block(prev_layer, current_size, conv_kernel, activation, dropout_rate, block_name, n)
        if current_size == last_layer_size: break
        current_size = max(last_layer_size, int(current_size / decay_factor))
        n += 1
    return prev_layer

def create_model():
    activation = 'relu'
    kernel_init = 'he_uniform'
    DropoutType = Dropout
    apply_batch_norm = True
    dense_dropout_rate = 0.5
    conv_dropout_rate = 0.5
    first_layer_size = 512
    last_layer_size = 16
    decay_factor = 1.4
    conv_decay_factor = 1.4
        
    model_name = "DeepTau2017v2p1"
    input_layer_tau = Input(name="input_tau", shape=X_taus_shape)
    input_layer_pfCand = Input(name="input_pfCand", shape=X_cells_pfCand_shape)
    input_layer_ele = Input(name="input_ele", shape=X_cells_ele_shape)
    input_layer_muon = Input(name="input_muon", shape=X_cells_muon_shape)
    
    reduced_pfCand = reduce_n_features_2d(input_layer_pfCand, 128, 16, conv_decay_factor, "pfCand", activation, conv_dropout_rate)
    reduced_ele = reduce_n_features_2d(input_layer_ele, 128, 16, conv_decay_factor, "ele", activation, conv_dropout_rate)
    reduced_muon = reduce_n_features_2d(input_layer_muon, 128, 16, conv_decay_factor, "muon", activation, conv_dropout_rate)
    
    conv_all_start = Concatenate(name="cell_concat", axis=3)([reduced_pfCand, reduced_ele, reduced_muon])
    cell_output_size = 32
    prev_layer = conv_block(conv_all_start, cell_output_size, (1, 1), activation, conv_dropout_rate, "all", 1)
    for n in range(5):
        if n == 4:
            cell_output_size = 16
        prev_layer = conv_block(prev_layer, cell_output_size, (3, 3), activation, conv_dropout_rate, "all_3x3", n+1)
    cells_flatten = Flatten(name="cells_flatten")(prev_layer)
        
    prev_layer = Concatenate(name="concat")([input_layer_tau, cells_flatten])
    current_size = first_layer_size
    n = 1
    while True:
        dense_layer = Dense(current_size, name="dense_%d" % n, kernel_initializer=kernel_init)(prev_layer)
        layer = dense_layer
        if apply_batch_norm:
            layer = BatchNormalization(name="batch_norm_%d" % n)(dense_layer)
        activation_layer = Activation(activation, name="activation_%d" % n)(layer)
        if dense_dropout_rate > 0:
            prev_layer = DropoutType(dense_dropout_rate, name="dropout_%d" % n)(activation_layer)
        else:
            prev_layer = activation_layer
        n += 1
        if current_size == last_layer_size: break
        current_size = max(last_layer_size, int(current_size / decay_factor))

    output_layer = Dense(n_outputs, name="dense_%d" % n)(prev_layer)
    softmax_output = Activation("softmax", name="main_output")(output_layer)

    model = Model([input_layer_tau, input_layer_pfCand, input_layer_ele, input_layer_muon], softmax_output, name="DeepTau2017v2")
    return model, model_name

In [5]:
def compile_model(model, learning_rate):
    opt = keras.optimizers.Adam(lr=learning_rate)
    #opt = keras.optimizers.Nadam(lr=learning_rate)
    #opt = keras.optimizers.SGD(lr=learning_rate, momentum=0.9, nesterov=True)
    #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
    metrics = ["accuracy", TauLosses.Le, TauLosses.Lmu, TauLosses.Ljet, TauLosses.sLe, TauLosses.sLmu, TauLosses.sLjet ]
    model.compile(loss=TauLosses.tau_crossentropy, optimizer=opt, metrics=metrics, weighted_metrics=metrics)

In [6]:
def close_file(f_name):
    file_objs = [ obj for obj in gc.get_objects() if ("TextIOWrapper" in str(type(obj))) and (obj.name == f_name)]
    for obj in file_objs:
        obj.close()

In [7]:
def run_training(train_suffix, model_name, data_loader, epoch, n_epochs):

    train_name = '%s_%s' % (model_name, train_suffix)
    
    cb_acc = ModelCheckpoint("%s_acc.hdf5" % train_name, monitor="val_acc", save_best_only=True,
                             save_weights_only=False, mode="max", verbose=1)
    
    cb_losses = []
    for loss_name in ["loss", "Le", "Lmu", "Ljet"]:
        cb_losses.append(ModelCheckpoint("%s_%s.hdf5" % (train_name, loss_name), monitor="val_%s" % loss_name,
                                         save_best_only=True, save_weights_only=False, mode="min", verbose=1))

    log_name = "%s.log" % train_name
    if os.path.isfile(log_name):
        close_file(log_name)
        os.remove(log_name)
    csv_log = CSVLogger(log_name, append=True)

    pbar = TQDMNotebookCallback(leave_outer=True, show_outer=True, leave_inner = True)
    callbacks = [pbar, csv_log, cb_acc, *cb_losses]
    fit_hist = model.fit_generator(data_loader.generator(True), validation_data=data_loader.generator(False),
                                   steps_per_epoch=data_loader.steps_per_epoch, validation_steps=data_loader.validation_steps,
                                   callbacks=callbacks, epochs=n_epochs, initial_epoch=epoch, verbose=0)

    model.save("%s_final.hdf5" % train_name)
    return fit_hist

In [8]:
loader = DataLoader('../tuples-v2-t1/training/part_*.root', 1000, n_cells, True)

In [9]:
TauLosses.SetSFs(2, 1, 5)
model, model_name = create_model()
compile_model(model, 1e-3)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_pfCand (InputLayer)       (None, 13, 13, 31)   0                                            
__________________________________________________________________________________________________
input_ele (InputLayer)          (None, 13, 13, 39)   0                                            
__________________________________________________________________________________________________
input_muon (InputLayer)         (None, 13, 13, 39)   0                                            
__________________________________________________________________________________________________
pfCand_conv_1 (Conv2D)          (None, 13, 13, 128)  4096        input_pfCand[0][0]               
__________________________________________________________________________________________________
ele_conv_1

__________________________________________________________________________________________________
pfCand_activation_7 (Activation (None, 13, 13, 16)   0           pfCand_batch_norm_7[0][0]        
__________________________________________________________________________________________________
ele_activation_7 (Activation)   (None, 13, 13, 16)   0           ele_batch_norm_7[0][0]           
__________________________________________________________________________________________________
muon_activation_7 (Activation)  (None, 13, 13, 16)   0           muon_batch_norm_7[0][0]          
__________________________________________________________________________________________________
pfCand_dropout_7 (Dropout)      (None, 13, 13, 16)   0           pfCand_activation_7[0][0]        
__________________________________________________________________________________________________
ele_dropout_7 (Dropout)         (None, 13, 13, 16)   0           ele_activation_7[0][0]           
__________

Non-trainable params: 6,220
__________________________________________________________________________________________________


In [10]:
fit_hist = run_training('step1', model_name, loader, 0, 10)


Epoch 00001: val_acc improved from -inf to 0.70708, saving model to DeepTau2017v2p1_step1_acc.hdf5

Epoch 00001: val_loss improved from inf to 0.85217, saving model to DeepTau2017v2p1_step1_loss.hdf5

Epoch 00001: val_Le improved from inf to 0.11305, saving model to DeepTau2017v2p1_step1_Le.hdf5

Epoch 00001: val_Lmu improved from inf to 0.06380, saving model to DeepTau2017v2p1_step1_Lmu.hdf5

Epoch 00001: val_Ljet improved from inf to 0.12721, saving model to DeepTau2017v2p1_step1_Ljet.hdf5



Epoch 00002: val_acc did not improve from 0.70708

Epoch 00002: val_loss improved from 0.85217 to 0.80429, saving model to DeepTau2017v2p1_step1_loss.hdf5

Epoch 00002: val_Le did not improve from 0.11305

Epoch 00002: val_Lmu did not improve from 0.06380

Epoch 00002: val_Ljet improved from 0.12721 to 0.11068, saving model to DeepTau2017v2p1_step1_Ljet.hdf5



Epoch 00003: val_acc did not improve from 0.70708

Epoch 00003: val_loss did not improve from 0.80429

Epoch 00003: val_Le did not improve from 0.11305

Epoch 00003: val_Lmu did not improve from 0.06380

Epoch 00003: val_Ljet improved from 0.11068 to 0.10868, saving model to DeepTau2017v2p1_step1_Ljet.hdf5



Epoch 00004: val_acc improved from 0.70708 to 0.70914, saving model to DeepTau2017v2p1_step1_acc.hdf5

Epoch 00004: val_loss improved from 0.80429 to 0.77023, saving model to DeepTau2017v2p1_step1_loss.hdf5

Epoch 00004: val_Le improved from 0.11305 to 0.11138, saving model to DeepTau2017v2p1_step1_Le.hdf5

Epoch 00004: val_Lmu improved from 0.06380 to 0.06219, saving model to DeepTau2017v2p1_step1_Lmu.hdf5

Epoch 00004: val_Ljet improved from 0.10868 to 0.10555, saving model to DeepTau2017v2p1_step1_Ljet.hdf5



Epoch 00005: val_acc did not improve from 0.70914

Epoch 00005: val_loss did not improve from 0.77023

Epoch 00005: val_Le did not improve from 0.11138

Epoch 00005: val_Lmu did not improve from 0.06219

Epoch 00005: val_Ljet improved from 0.10555 to 0.10415, saving model to DeepTau2017v2p1_step1_Ljet.hdf5



Epoch 00006: val_acc did not improve from 0.70914

Epoch 00006: val_loss did not improve from 0.77023

Epoch 00006: val_Le did not improve from 0.11138

Epoch 00006: val_Lmu did not improve from 0.06219

Epoch 00006: val_Ljet improved from 0.10415 to 0.10103, saving model to DeepTau2017v2p1_step1_Ljet.hdf5



Epoch 00007: val_acc did not improve from 0.70914

Epoch 00007: val_loss improved from 0.77023 to 0.74635, saving model to DeepTau2017v2p1_step1_loss.hdf5

Epoch 00007: val_Le improved from 0.11138 to 0.11107, saving model to DeepTau2017v2p1_step1_Le.hdf5

Epoch 00007: val_Lmu improved from 0.06219 to 0.05773, saving model to DeepTau2017v2p1_step1_Lmu.hdf5

Epoch 00007: val_Ljet improved from 0.10103 to 0.09764, saving model to DeepTau2017v2p1_step1_Ljet.hdf5


KeyboardInterrupt: 